<a href="https://colab.research.google.com/github/Greg9998/Demo111/blob/master/HeartInHearth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
# clear all
from IPython import get_ipython
get_ipython().magic('reset -sf')

Libraries, data read, preprocessing and subsets generation

In [119]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import sklearn.metrics
from sklearn.preprocessing import OrdinalEncoder #Categorical replacement
from sklearn.model_selection import train_test_split #random train-test subsets

#Settings the warnings to be ignored
warnings.filterwarnings('ignore')

#Read data
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/heart_disease.csv')
data0 = data.copy()

#Descriptive statistics function
def summary(df):
    print('data shape: ',df.shape)
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['missing'] = df.isnull().sum().values
    summ['%miss'] = df.isnull().sum().values / len(df)
    summ['unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['Quant1'] = df.quantile(0.25)
    summ['median'] = df.quantile(0.5)
    summ['Quant3'] = df.quantile(0.75)
    summ['max'] = desc['max'].values
    summ['mean'] = df.mean()
    summ['std'] = df.std()
    summ['skew'] = df.skew()
    summ['kurt'] = df.kurt()
    return summ

#Missing values removal
data.dropna(inplace=True)

#Categorical replacement into int
## naturaly progression integers
enc = OrdinalEncoder(categories=[['uneducated','primaryschool','graduate','postgraduate']])
enc.fit(data[['education']])
data[['education']] = enc.transform(data[['education']])

## binary vals
enc = OrdinalEncoder()
enc.fit(data[['Gender','prevalentStroke','Heart_ stroke']])
data[['Gender','prevalentStroke','Heart_ stroke']] = enc.transform(data[['Gender','prevalentStroke','Heart_ stroke']])

#Outliers removal
outliersThreshold = 0.005

q_low = data.quantile(outliersThreshold)
q_hi = data.quantile(1-outliersThreshold)
logic = True
for col in data.columns:
  logic = logic & (data[col]<=q_hi[col]) & (data[col]>=q_low[col])
data2 = data[logic]

#delete unnecessary variables
del enc, col, logic, q_low, q_hi

#Train-test-validation split function
def train_test_valid_split(target,data,rTR=60,rTE=20,rVA=20):
  rTT=1-rTR/(rTR+rTE+rVA) #remaining fraction after train subset generation
  rTV=rTE/(rTE+rVA) #test vs validation subset fraction
  x_train, x_test, y_train, y_test = train_test_split(target, data, test_size=rTT, random_state=12)
  x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size=rTV, random_state=13)
  return x_train, x_test, x_valid, y_train, y_test, y_valid

#Split data2 into training, testing and validation subsets
## choose features
cx = data2[['age', 'sysBP', 'glucose']]

## extract target
x = data2.drop('Heart_ stroke', axis = 1)
y = data2['Heart_ stroke']

## Split data2 (with 'c' - subsets with only chosen features)
xc_train, xc_test, xc_valid, yc_train, yc_test, yc_valid = train_test_valid_split(cx,y,60,20,20)
x_train, x_test, x_valid, y_train, y_test, y_valid = train_test_valid_split(x,y)

#sklearn.metrics accuracy and precision print function
def sklmetrics(y_train,y_pred_train,y_valid,y_pred_valid):
  acc_train = sklearn.metrics.accuracy_score(y_train, y_pred_train)
  acc_valid = sklearn.metrics.accuracy_score(y_valid, y_pred_valid)
  prec_train = sklearn.metrics.precision_score(y_train, y_pred_train)
  prec_valid = sklearn.metrics.precision_score(y_valid, y_pred_valid)
  print(f'Train accuracy = {acc_train:.7f}\nValid accuracy = {acc_valid:.7f}')
  print(f'Train precision = {prec_train:.7f}\nValid precision = {prec_valid:.7f}')

summary(data2)

data shape:  (3441, 16)


,data type,missing,%miss,unique,min,Quant1,median,Quant3,max,mean,std,skew,kurt
Gender,float64,0,0.0,2,0.00,0.0,0.0,1.00,1.00,0.446382,0.497189,0.215811,-1.954562
age,int64,0,0.0,35,34.00,42.0,49.0,56.00,68.00,49.473409,8.445031,0.232585,-0.999850
education,float64,0,0.0,4,0.00,0.0,1.0,2.00,3.00,0.993316,1.027648,0.671072,-0.755260
currentSmoker,int64,0,0.0,2,0.00,0.0,0.0,1.00,1.00,0.487940,0.499927,0.048277,-1.998831
cigsPerDay,float64,0,0.0,29,0.00,0.0,0.0,20.00,43.00,8.880267,11.568535,1.090714,0.207265
BPMeds,float64,0,0.0,2,0.00,0.0,0.0,0.00,1.00,0.027027,0.162186,5.835878,32.076111
prevalentStroke,float64,0,0.0,2,0.00,0.0,0.0,0.00,1.00,0.005812,0.076027,13.007819,167.300589
prevalentHyp,int64,0,0.0,2,0.00,0.0,0.0,1.00,1.00,0.303110,0.459669,0.857160,-1.266013
diabetes,int64,0,0.0,2,0.00,0.0,0.0,0.00,1.00,0.021215,0.144121,6.648097,42.221730
totChol,float64,0,0.0,214,150.00,207.0,234.0,264.00,372.00,236.859924,41.526310,0.359809,-0.132466


In [120]:
#-----------------------------classification-----------------------------------
from sklearn.model_selection import cross_validate

from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

names = [
    "Nearest_Neighbors",
    "Linear_SVM",
    "RBF_SVM",
    #"Gaussian_Process",
    "Decision_Tree",
    "Random_Forest",
    "Neural_Net",
    "AdaBoost",
    "Naive_Bayes",
    "QDA",
]

rows = ["Mean_fit_time","Train score", "Train recall","Train accuracy","Train precision",
        "Valid score", "Valid recall","Valid accuracy","Valid precision"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025, random_state=42),
    SVC(gamma=2, C=1, random_state=42),
    #GaussianProcessClassifier(1.0 * RBF(1.0), random_state=42),
    DecisionTreeClassifier(max_depth=5, random_state=42),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, random_state=42),
    MLPClassifier(alpha=1, max_iter=1000, random_state=42),
    AdaBoostClassifier(random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

#Function for score, accuracy and precision, total recall, etc...
def TotalRecall(clf,x_train,y_train,x_valid,y_valid):
  y_pred_train = clf.predict(x_train)
  y_pred_valid = clf.predict(x_valid)
  clf_results = cross_validate(clf, x_train, y_train, cv=3)
  return [
      (clf_results['fit_time']).mean(), #mean_fit_time
      clf.score(x_train, y_train), #Train score
      sklearn.metrics.recall_score(y_train, y_pred_train), #Train recall
      sklearn.metrics.accuracy_score(y_train, y_pred_train), #Train accuracy
      sklearn.metrics.precision_score(y_train, y_pred_train), #Train precision
      clf.score(x_valid, y_valid), #Valid score
      sklearn.metrics.recall_score(y_valid, y_pred_valid), #Valid recall
      sklearn.metrics.accuracy_score(y_valid, y_pred_valid), #Valid accuracy
      sklearn.metrics.precision_score(y_valid, y_pred_valid), #Valid precision
  ]

# iterate over classifiers for all data
scoreAll = pd.DataFrame([], index=rows, columns=names)
for name, clf in zip(names, classifiers):
  clf = make_pipeline(StandardScaler(), clf)
  clf.fit(x_train, y_train)
  scoreAll[name]=TotalRecall(clf,x_train,y_train,x_valid,y_valid)

scoreAll

,Nearest_Neighbors,Linear_SVM,RBF_SVM,Decision_Tree,Random_Forest,Neural_Net,AdaBoost,Naive_Bayes,QDA
Mean_fit_time,0.031625,0.029438,0.120707,0.012894,0.024798,3.620484,0.151198,0.005334,0.008000
Train score,0.890504,0.853682,0.996609,0.875000,0.857558,0.862403,0.864341,0.826550,0.834302
Train recall,0.390728,0.000000,0.976821,0.172185,0.026490,0.076159,0.135762,0.208609,0.175497
Train accuracy,0.890504,0.853682,0.996609,0.875000,0.857558,0.862403,0.864341,0.826550,0.834302
Train precision,0.737500,0.000000,1.000000,0.866667,1.000000,0.821429,0.683333,0.346154,0.363014
Valid score,0.822674,0.867733,0.867733,0.859012,0.866279,0.867733,0.850291,0.838663,0.841570
Valid recall,0.120879,0.000000,0.000000,0.065934,0.000000,0.032967,0.065934,0.164835,0.109890
Valid accuracy,0.822674,0.867733,0.867733,0.859012,0.866279,0.867733,0.850291,0.838663,0.841570
Valid precision,0.207547,0.000000,0.000000,0.333333,0.000000,0.500000,0.250000,0.300000,0.263158


In [121]:
# iterate over classifiers for XC data
scoreXC = pd.DataFrame([], index=rows, columns=names)
for name, clf in zip(names, classifiers):
  clf = make_pipeline(StandardScaler(), clf)
  clf.fit(xc_train, yc_train)
  scoreXC[name]=TotalRecall(clf,xc_train,yc_train,xc_valid,yc_valid)

scoreXC

,Nearest_Neighbors,Linear_SVM,RBF_SVM,Decision_Tree,Random_Forest,Neural_Net,AdaBoost,Naive_Bayes,QDA
Mean_fit_time,0.011946,0.016251,0.061369,0.005538,0.025490,1.405772,0.116841,0.004353,0.004156
Train score,0.884205,0.853682,0.862888,0.863857,0.861434,0.853682,0.857558,0.838178,0.840116
Train recall,0.377483,0.000000,0.062914,0.142384,0.052980,0.000000,0.052980,0.158940,0.102649
Train accuracy,0.884205,0.853682,0.862888,0.863857,0.861434,0.853682,0.857558,0.838178,0.840116
Train precision,0.690909,0.000000,1.000000,0.661538,1.000000,0.000000,0.666667,0.375000,0.344444
Valid score,0.811047,0.867733,0.863372,0.847384,0.866279,0.867733,0.861919,0.837209,0.843023
Valid recall,0.109890,0.000000,0.000000,0.054945,0.021978,0.000000,0.043956,0.142857,0.109890
Valid accuracy,0.811047,0.867733,0.863372,0.847384,0.866279,0.867733,0.861919,0.837209,0.843023
Valid precision,0.169492,0.000000,0.000000,0.208333,0.400000,0.000000,0.333333,0.276596,0.270270
